In [3]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(0)

# Number of organizations
n = 100

# Generate random data
data = {
    'organization_id': range(1, n + 1),
    'low_density_subs': np.random.randint(0, 10, n),
    'high_density_subs': np.random.randint(0, 10, n),
    'physical_servers': np.random.randint(1, 10, n),
    'virtual_machines': np.random.randint(0, 50, n),
    'number_of_vms_cloud': np.random.randint(0, 20, n),
    'sockets': np.random.randint(1, 10, n),
    'virtualization_hosts': np.random.randint(0, 5, n)
}

# Create DataFrame
df = pd.DataFrame(data)

#   We are classifying organizations as compliant or non-compliant based on the following rules:
#   https://www.suse.com/products/terms_and_conditions.pdf
def classify_compliance(row):
    # Accessing the columns from the row
    low_density_subs = row['low_density_subs']
    high_density_subs = row['high_density_subs']
    physical_servers = row['physical_servers']
    virtual_machines = row['virtual_machines']
    number_of_vms_cloud = row['number_of_vms_cloud']
    sockets = row['sockets']
    virtualization_hosts = row['virtualization_hosts']

    # Rule 1: Each physical server should have at least one subscription (either low or high density).
    total_subs = low_density_subs + high_density_subs
    if total_subs < physical_servers:
        return 0
    
    # Rule 2: If there are virtualization hosts, there should be at least one high-density subscription.
    if virtualization_hosts > 0 and high_density_subs < 1:
        return 0
    
    # Rule 2: For up to 2 VMs, at least one low-density subscription is required.  
    if virtual_machines <= 2 and low_density_subs < 1:
        return 0
    
    # Rule 3: Each high-density subscription covers unlimited VMs and 1-2 sockets.
    # Each low-density subscription covers 1-2 sockets or 1-2 VMs.
    # Checking if the number of sockets is covered by the total subscriptions.
    if sockets > (total_subs * 2):
        return 0
    
    return 1  # If none of the above conditions are met, the organization is compliant.

# Applying the function to each row of the DataFrame
df['compliance'] = df.apply(classify_compliance, axis=1)

# Now df['compliance'] will have the compliance status for each organization

# Save to CSV
df.to_csv('organizations.csv', index=False)
